# Chapter 09: -4- Converter.py with importlib and inspect modules
2020-11-17

Content:
1. UML diagram of the unitconver package
2. setup.py and CLI.py for the entry point (and argParse module)
3. UnitTable.py and the unit_tables directory (and lambda function)
4. **Converter.py (and importlib module + inspect module)**
5. Putting all of them together!

## 幾個重要的概念：
1. **import importlib**: 如何載入一個 py 檔的 module
2. **getattr()** 如何從一個 module 中，取出其中定義的一個 class 的 type，然後利用在 **type 後馬上補上()** 就可 instantiate 一個相對應的物件
3. 取得物件之後，會將其定義在這物件中，內藏在 dictionary 中的 lambda function 給取出來作運算。
4. **inspect module** 在我們執要環境中的 modules, classes, methods, functions,...等，一定都是從 code 來的，我們會想要知道其相對應的 code 為何？

In [1]:
import os
course_directory = 'D:\\Google雲端硬碟\\GettingStartedWithPythonAndRaspberryPi-book_release'
os.chdir(course_directory + "/Chapter09/unitconverter")

In [2]:
!unitconvert length list

Unit table length can convert between the units:
um
cm
mil
mm
yd
mi
ft
in
m


In [3]:
!unitconvert length list -m

Unit table length can convert between the units:
mil (x * 39370.078740158)
cm (x * 100.0)
mi (x * 0.00062137)
m (base unit)
mm (x * 1000.0)
ft (x * 3.2808)
in (x * 39.370)
um (x * 1000000.0)
yd (x * 1.0936)


In [4]:
!unitconvert length convert 1 mi m yd ft in cm

1.000000 mi = 1609.347088 m
1.000000 mi = 1759.981975 yd
1.000000 mi = 5279.945926 ft
1.000000 mi = 63359.994850 in
1.000000 mi = 160934.708789 cm


In [ ]:
'''
這個 CLI 的程式是在 setup 之後，在 console 可以要說明性的 help，長得如下：
===
usage: unitconvert [-h] TABLE {list,convert} ...

Tool for converting units

positional arguments:
  TABLE           Unit table to use in conversion
  {list,convert}  operation to be performed

optional arguments:
  -h, --help      show this help message and exit
  
'''

import argparse
import inspect
from unitconverter.Converter import get_table, convert_units


def run_cli():
    parser = argparse.ArgumentParser(description='Tool for converting units')

    parser.add_argument(
        'table',
        metavar='TABLE',
        action='store',
        type=str,
        help='Unit table to use in conversion'
    )

    """
    這是 unitconvert 以及上述 table 之後，第二層要解碼的部份，
    這又分為兩個：
    1. list
    2. convert
    我們安排一個 which 變數來記錄是哪一個。
    """
    subparsers = parser.add_subparsers(help='operation to be performed')

    '''
    以下考慮 -m, --method 的選項，要作什麼事
    
    Unit table energy can convert between the units:
    kcal (x * 0.00023884589663)
    cal (x * 0.23900573614)
    btu (x * 0.00094781707775)
    ev (x * 6241506480000000000.0)
    wh (x * 0.00027777777778)
    j (base unit)
    hph (x * 3.7250614123e-7)
    '''
    
    list_table_parser = subparsers.add_parser('list')
    list_table_parser.set_defaults(which='list')

    list_table_parser.add_argument(
        '-m', '--method',
        action='store_true',
        help='Also output the conversion method from the base unit'
    )
    
    '''
    以下考慮 convert 時，要作什麼事
    
    usage: unitconvert TABLE convert [-h] VALUE FROM TO [TO ...]

    positional arguments:
      VALUE       The value to convert
      FROM        Unit to convert from
      TO          Unit(s) to convert to

    optional arguments:
      -h, --help  show this help message and exit
    '''

    conversion_parser = subparsers.add_parser('convert')
    conversion_parser.set_defaults(which='convert')

    conversion_parser.add_argument(
        'value',
        metavar='VALUE',
        type=float,
        action='store',
        help='The value to convert'
    )

    conversion_parser.add_argument(
        'from_unit',
        metavar='FROM',
        action='store',
        type=str,
        help='Unit to convert from'
    )

    conversion_parser.add_argument(
        'to_units',
        metavar='TO',
        action='store',
        nargs='+', # 個數可以有很多個，但至少會有一個
        type=str,
        help='Unit(s) to convert to'
    )

    """
    以下這個指令 parser.parse_args() 是最重要的，
    將 CLI 輸入的參數按指定的名稱取出，
    然後，就可以作判斷，要作怎樣的反應。
    """
    props = parser.parse_args()

    if props.which == 'list':
        _run_unit_list(props)
    elif props.which == 'convert':
        _run_conversion(props)


def _run_unit_list(props):
    """
    Runs the command line interface for unit listing mode.

    @param props Properties parsed by argparse
    """

    table = get_table(props.table)
    print('Unit table %s can convert between the units:' % props.table)
    for unit in table.get_units():
        if props.method:
            if unit == table.base_unit:
                formula = 'base unit'
            else:
                conversion = inspect.getsource(table.from_base_unit[unit])
                """
                self.from_base_unit['radian'] = lambda x: x * 0.0174532925
                """
                formula = conversion[conversion.index(':') + 1:conversion.index('\n')].strip() # 這是要將一開頭的 [lambda x:] 給去掉
            print('%s (%s)' % (unit, formula))
        else:
            print(unit)


def _run_conversion(props):
    """
    Runs the command line interface for unit conversion mode.

    @param props Properties parsed by argparse
    """

    results = convert_units(table_name=props.table,
                            value=props.value,
                            value_unit=props.from_unit,
                            targets=props.to_units)

    for result in results:
        print('%f %s = %f %s' % (props.value, props.from_unit,
                                 result['converted_value'],
                                 result['dest_unit']))


## 將重點放在 {list, convert} 之後的處理：
```python:
    props = parser.parse_args()

    if props.which == 'list':
        _run_unit_list(props)
    elif props.which == 'convert':
        _run_conversion(props)
```

## 来看看 unitconverter 的核心程式 Converter.py
 
 * get_table: 回傳一個 table_name 所指定的 unit_table 物件
 * convert_units: 把用戶指定的數值轉換為以指定單位表示的形式

In [5]:
'''
以下是 Converter.py 內容
'''
import importlib

def get_table(table_name):
    """
    Returns an instance of a unit table given the name of the table.

    @param table_name Name of table to retrieve
    @return UnitTable instance
    """

    converter_module = importlib.import_module('unitconverter.unit_tables.%s' % table_name)
    converter_class = getattr(converter_module, table_name)()
    return converter_class


def convert_units(table_name, value, value_unit, targets):
    """
    Converts a given value in a unit to a set of target units.

    @param table_name Name of table units are contained in
    @param value Value to convert
    @param value_unit Unit value is currently in
    @param targets List of units to convert to
    @return List of conversions
    """

    table = get_table(table_name)

    results = list() # 之後會存入每筆為 dictionary 的結果
    for target in targets:
        result = {'dest_unit': target}
        try:
            result['converted_value'] = table.convert(value_unit, target, value)
        except ValueError:
            continue
        results.append(result)

    return results

In [6]:
from unitconverter.Converter import get_table, convert_units
## 之後就直接使用 get_table, convert_units，而不再提是從 Converter.ps 中所定義的

In [7]:
'''
先來看 Converter.py 中比較簡單的：convert_units()
'''
convert_units??

In [8]:
'''
def convert_units(table_name, value, value_unit, targets):
'''
convert_units('time', 1, 'd', ['h', 'm'])

[{'dest_unit': 'h', 'converted_value': 24.0},
 {'dest_unit': 'm', 'converted_value': 1440.0}]

In [9]:
'''
另外一個 get_table 就比較需要說明：
from unitconverter.Converter import get_table, convert_units
'''
?get_table

## (非常重要！) Converter.py 中的 get_table() 涉及到：
1. 在程式裏面，run-time 讀取另一個 module 程式中的某段相關程式碼
2. 針對定義一個類別的 type，如何 instantiate 一個相對應的物件：如 time 的 unitTable
3. 再從這個 time 的 unitTable 提取出 lambda function 作 list 或是 convert 之用。

```python:
def get_table(table_name):
    """
    Returns an instance of a unit table given the name of the table.

    @param table_name Name of table to retrieve
    @return UnitTable instance
    """
    
    """
    這個 importlib 是從目錄中 import 指定的 module
    然後，instantiate 一個這個類別的一個物件，
    最後回傳。
    """

    converter_module = importlib.import_module('unitconverter.unit_tables.%s' % table_name)
    converter_class = getattr(converter_module, table_name)()
    return converter_class
```

##  **get_table(table_name)** code 中，我們專注以下兩行指令：
```python:
    converter_module = importlib.import_module('unitconverter.unit_tables.%s' % table_name)
    converter_class = getattr(converter_module, table_name)()
```

In [10]:
'''
試一下 'time'：回傳是一個 time class 的 object
'''
get_table('time')

In [11]:
type(get_table('time'))

unitconverter.unit_tables.time.time

In [12]:
'''
我們針對 get_table() 回傳的值 作 help()
'''
help(get_table('time'))

Help on time in module unitconverter.unit_tables.time object:

class time(unitconverter.UnitTable.UnitTable)
 |  A class to manage conversion of units for the same quantity (e.g. length,
 |  volume, mass)
 |  
 |  Method resolution order:
 |      time
 |      unitconverter.UnitTable.UnitTable
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from unitconverter.UnitTable.UnitTable:
 |  
 |  can_convert(self, unit)
 |      Checks to see if this table can convert a given unit.
 |      
 |      @param unit Name of unit to check
 |      @return Boolean indicating if this unit can be converted
 |  
 |  convert(self, source, dest, value)
 |      Converts a value in one unit to another.
 |      
 |      @param source The source unit
 |      @param dest The destination unit
 |      @param value T

In [13]:
'''
time 這個表格的 object，一共定義幾個單位
'''
get_table('time').get_units()

{'d', 'h', 'm', 'ms', 'ns', 's', 'us', 'y'}

In [14]:
get_table('time').can_convert("m")

True

In [15]:
get_table('time').can_convert("j")

False

In [16]:
get_table('time')

In [17]:
'''
對於以上所 instantiated 出來的 time object，
有那些最重要的 attributes？
'''
get_table('time').__dict__

{'base_unit': 's',
 'to_base_unit': {'m': <function unitconverter.unit_tables.time.time.__init__.<locals>.<lambda>(x)>,
  'h': <function unitconverter.unit_tables.time.time.__init__.<locals>.<lambda>(x)>,
  'd': <function unitconverter.unit_tables.time.time.__init__.<locals>.<lambda>(x)>,
  'y': <function unitconverter.unit_tables.time.time.__init__.<locals>.<lambda>(x)>,
  'ms': <function unitconverter.unit_tables.time.time.__init__.<locals>.<lambda>(x)>,
  'us': <function unitconverter.unit_tables.time.time.__init__.<locals>.<lambda>(x)>,
  'ns': <function unitconverter.unit_tables.time.time.__init__.<locals>.<lambda>(x)>},
 'from_base_unit': {'m': <function unitconverter.unit_tables.time.time.__init__.<locals>.<lambda>(x)>,
  'h': <function unitconverter.unit_tables.time.time.__init__.<locals>.<lambda>(x)>,
  'd': <function unitconverter.unit_tables.time.time.__init__.<locals>.<lambda>(x)>,
  'y': <function unitconverter.unit_tables.time.time.__init__.<locals>.<lambda>(x)>,
  'ms': 

##  Converter.py 這僅含 function 定義的 module 中，
其 **get_table()** 如何載入一個 py 檔的 module，\
在其為一個 class 時，如何 instantiate 一個相對應的 object？

## 眉角在哪？

怎麼樣能夠 動態性 (run-time) 地 import module，
然後又 instantiate 該類別的一個 object

In [21]:
'''
使用 importlib.import_module() 將一個 py 結尾的檔案讀入
'''
converter_module = importlib.import_module('unitconverter.unit_tables.%s' % "time")
converter_module

<module 'unitconverter.unit_tables.time' from 'D:\\Google雲端硬碟\\GettingStartedWithPythonAndRaspberryPi-book_release\\Chapter09\\unitconverter\\unitconverter\\unit_tables\\time.py'>

In [22]:
print(converter_module)

<module 'unitconverter.unit_tables.time' from 'D:\\Google雲端硬碟\\GettingStartedWithPythonAndRaspberryPi-book_release\\Chapter09\\unitconverter\\unitconverter\\unit_tables\\time.py'>


In [23]:
'''
這 module 有很多的屬性，可以用 .__dict__ 讀出來
'''
converter_module.__dict__

{'__name__': 'unitconverter.unit_tables.time',
 '__doc__': None,
 '__package__': 'unitconverter.unit_tables',
 '__loader__': <_frozen_importlib_external.SourceFileLoader at 0x126a5050588>,
 '__spec__': ModuleSpec(name='unitconverter.unit_tables.time', loader=<_frozen_importlib_external.SourceFileLoader object at 0x00000126A5050588>, origin='D:\\Google雲端硬碟\\GettingStartedWithPythonAndRaspberryPi-book_release\\Chapter09\\unitconverter\\unitconverter\\unit_tables\\time.py'),
 '__file__': 'D:\\Google雲端硬碟\\GettingStartedWithPythonAndRaspberryPi-book_release\\Chapter09\\unitconverter\\unitconverter\\unit_tables\\time.py',
 '__cached__': 'D:\\Google雲端硬碟\\GettingStartedWithPythonAndRaspberryPi-book_release\\Chapter09\\unitconverter\\unitconverter\\unit_tables\\__pycache__\\time.cpython-37.pyc',
 '__builtins__': {'__name__': 'builtins',
  '__doc__': "Built-in functions, exceptions, and other objects.\n\nNoteworthy: None is the `nil' object; Ellipsis represents `...' in slices.",
  '__package__'

In [25]:
'''
這屬性的條列很像是 dictionary，可以用 key 來查詢
得到的結果是一個 class
'''
a = converter_module.__dict__["time"]
a

unitconverter.unit_tables.time.time

In [26]:
print(a)

<class 'unitconverter.unit_tables.time.time'>


In [27]:
type(a)

type

In [28]:
a()

In [29]:
'''
我們可以直接 instantiate 一個這樣的物件
注意後面的 (), 這與我們要 instantiate 一個物件用 time.time() 一樣
'''
converter_module.__dict__["time"]()

In [30]:
'''
我們將上述的流程寫成指令

以下是課本的寫法，用 converter_class，
但~
其實回傳的是 instance，並不是 class
而是一個 type
'''
converter_class = getattr(converter_module, "time")()
converter_class

In [31]:
type(converter_class)
# 它是一個 object

unitconverter.unit_tables.time.time

In [32]:
'''
這樣所建置出來的 object 內含的 attributes 可用 __dict__ 得到：
'''
converter_class.__dict__

{'base_unit': 's',
 'to_base_unit': {'m': <function unitconverter.unit_tables.time.time.__init__.<locals>.<lambda>(x)>,
  'h': <function unitconverter.unit_tables.time.time.__init__.<locals>.<lambda>(x)>,
  'd': <function unitconverter.unit_tables.time.time.__init__.<locals>.<lambda>(x)>,
  'y': <function unitconverter.unit_tables.time.time.__init__.<locals>.<lambda>(x)>,
  'ms': <function unitconverter.unit_tables.time.time.__init__.<locals>.<lambda>(x)>,
  'us': <function unitconverter.unit_tables.time.time.__init__.<locals>.<lambda>(x)>,
  'ns': <function unitconverter.unit_tables.time.time.__init__.<locals>.<lambda>(x)>},
 'from_base_unit': {'m': <function unitconverter.unit_tables.time.time.__init__.<locals>.<lambda>(x)>,
  'h': <function unitconverter.unit_tables.time.time.__init__.<locals>.<lambda>(x)>,
  'd': <function unitconverter.unit_tables.time.time.__init__.<locals>.<lambda>(x)>,
  'y': <function unitconverter.unit_tables.time.time.__init__.<locals>.<lambda>(x)>,
  'ms': 

In [33]:
converter_class.base_unit

's'

## 接下來看： lambda function 的使用

它是一個 function，但這個 function 有一個 key 可當作參數，調整為面向不同的「參數」。\
取決於不同的 source，叫作同名的 self.to_base_unit(value) 會有不同的定義 \
self.to_base_unit\[source\](value)

In [34]:
'''
我們看到 converter_class.__dict__ 是一個 dict，
對於 converter_class 這個物件的各個 attribute，
它都會有一些特性，藏在以 dictionary 的 (key: valuse) 中。

而如果用 'to_base_unit' 為 key 去查詢，發現它的值也是一個 dict，
裏面記的 key 是各種單位，而 value 則是相對應其轉換至 base 的公式，

'to_base_unit': {'m': <function unitconverter.unit_tables.time.time.__init__.<locals>.<lambda>(x)>,
  'h': <function unitconverter.unit_tables.time.time.__init__.<locals>.<lambda>(x)>,
  'd': <function unitconverter.unit_tables.time.time.__init__.<locals>.<lambda>(x)>,
  'y': <function unitconverter.unit_tables.time.time.__init__.<locals>.<lambda>(x)>,
  'ms': <function unitconverter.unit_tables.time.time.__init__.<locals>.<lambda>(x)>,
  'us': <function unitconverter.unit_tables.time.time.__init__.<locals>.<lambda>(x)>,
  'ns': <function unitconverter.unit_tables.time.time.__init__.<locals>.<lambda>(x)>},
  
這個後面的  ["d"] 從上面的 .__dict__ 可看出，是一個 dictionary 的查詢概念
'''
converter_class.to_base_unit["d"]
# 他的查詢結果就是一個 lambda function

<function unitconverter.unit_tables.time.time.__init__.<locals>.<lambda>(x)>

In [36]:
!unitconvert time list -m

Unit table time can convert between the units:
d (x / 86400.0)
us (x * 1000000.0)
h (x / 3600.0)
s (base unit)
y (x / 3.15569e7)
ms (x * 1000.0)
ns (x * 1000000000.0)
m (x / 60.0)


## inspect module in python
**The *inspect* module provides several useful functions to help get information about live objects such as modules, classes, methods, functions, tracebacks, frame objects, and code objects. **

在我們執要環境中的 modules, classes, methods, functions,...等，一定都是從 code 來的，我們會想要知道其相對應的 code 為何？ \
→ inspect module

## inspect — Inspect live objects
https://docs.python.org/3/library/inspect.html


## Python: Using the inspect module to get line numbers and other information

這個教學影片講解程式執行出現錯誤時 traceboack 的資訊是怎麼來的？
https://www.youtube.com/watch?v=myTz-ZDkO6Q

In [37]:
%%HTML
<iframe width="560" height="315" src="https://www.youtube.com/embed/myTz-ZDkO6Q" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>

In [38]:
type(converter_class.to_base_unit["d"])

function

In [41]:
# 進去看相對應的程式碼
import inspect
inspect.getsource(converter_class.to_base_unit["d"])

"        self.to_base_unit['d'] = lambda x: x * 86400.0\n"

In [42]:
inspect.getsource(converter_class.convert)

'    def convert(self, source, dest, value):\n        """\n        Converts a value in one unit to another.\n\n        @param source The source unit\n        @param dest The destination unit\n        @param value The value to convert\n        @return The value in the destination unit\n        """\n\n        if not (self.can_convert(source) and self.can_convert(dest)):\n            raise ValueError(\'Cannot convert given units\')\n\n        if source != self.base_unit:\n            value = self.to_base_unit[source](value)\n\n        if dest != self.base_unit:\n            value = self.from_base_unit[dest](value)\n\n        return value\n'

In [43]:
print(inspect.getsource(converter_class.convert))

    def convert(self, source, dest, value):
        """
        Converts a value in one unit to another.

        @param source The source unit
        @param dest The destination unit
        @param value The value to convert
        @return The value in the destination unit
        """

        if not (self.can_convert(source) and self.can_convert(dest)):
            raise ValueError('Cannot convert given units')

        if source != self.base_unit:
            value = self.to_base_unit[source](value)

        if dest != self.base_unit:
            value = self.from_base_unit[dest](value)

        return value



In [44]:
type(converter_class.base_unit)

str

In [45]:
# 但如果是 str 的話，就沒辦法看原始碼了
inspect.getsource(converter_class.base_unit)

TypeError: module, class, method, function, traceback, frame, or code object was expected, got str

In [37]:
%%HTML
<iframe width="560" height="315" src="https://www.youtube.com/embed/myTz-ZDkO6Q" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>

In [46]:
'''
一旦取得 lambda function 之後，後面只要用 () 即可置入函數的參數
然後，得到答案。
'''
converter_class.to_base_unit["d"](1)

86400.0

### 說明例：將200 wh 的能量分別轉換為 j, hph的形式

In [47]:
get_table('energy')

In [48]:
get_table('energy').__dict__

{'base_unit': 'j',
 'to_base_unit': {'ev': <function unitconverter.unit_tables.energy.energy.__init__.<locals>.<lambda>(x)>,
  'cal': <function unitconverter.unit_tables.energy.energy.__init__.<locals>.<lambda>(x)>,
  'kcal': <function unitconverter.unit_tables.energy.energy.__init__.<locals>.<lambda>(x)>,
  'hph': <function unitconverter.unit_tables.energy.energy.__init__.<locals>.<lambda>(x)>,
  'wh': <function unitconverter.unit_tables.energy.energy.__init__.<locals>.<lambda>(x)>,
  'btu': <function unitconverter.unit_tables.energy.energy.__init__.<locals>.<lambda>(x)>},
 'from_base_unit': {'ev': <function unitconverter.unit_tables.energy.energy.__init__.<locals>.<lambda>(x)>,
  'cal': <function unitconverter.unit_tables.energy.energy.__init__.<locals>.<lambda>(x)>,
  'kcal': <function unitconverter.unit_tables.energy.energy.__init__.<locals>.<lambda>(x)>,
  'hph': <function unitconverter.unit_tables.energy.energy.__init__.<locals>.<lambda>(x)>,
  'wh': <function unitconverter.unit_

In [49]:
get_table('energy').base_unit

'j'

In [51]:
get_table('energy').to_base_unit["cal"]

<function unitconverter.unit_tables.energy.energy.__init__.<locals>.<lambda>(x)>

In [52]:
# 好奇偷看一下
inspect.getsource(get_table('energy').to_base_unit["cal"])

"        self.to_base_unit['cal'] = lambda x: x / 0.23900573614\n"

In [53]:
get_table('energy').to_base_unit["cal"](1)

4.183999999959164

In [54]:
'''
當然，當作繼承 UnitTable 類別的各個 table，也同時繼承原父輩的各定義的 method，像是：
'''
get_table('energy').get_units()

{'btu', 'cal', 'ev', 'hph', 'j', 'kcal', 'wh'}

In [55]:
convert_units('energy', 200, 'wh', ['j', 'hph'])

[{'dest_unit': 'j', 'converted_value': 719999.99999424},
 {'dest_unit': 'hph', 'converted_value': 0.2682044216834544}]

In [56]:
convert_units('time', 1, 'd', ['h', 'm'])

[{'dest_unit': 'h', 'converted_value': 24.0},
 {'dest_unit': 'm', 'converted_value': 1440.0}]